## Conversational BOT with History

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")



In [9]:
print("Chatbot is ready! Type 'quit' to stop.\n")

chat_history_ids = None  # stores past conversation context

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Chatbot session ended.")
        break

    # Encode user input with end-of-sequence token
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Concatenate new input with chat history (if any)
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    # Generate response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,  # creativity
        top_p=0.9,        # diversity
        do_sample=True
    )

    # ✅ Decode only the new tokens (ignore old history)
    bot_reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"User: {user_input}")
    print(f"Bot: {bot_reply}\n")

Chatbot is ready! Type 'quit' to stop.

User: Hi 
Bot: Hi, welcome to the sub!

User: do you like pizza
Bot: I love pizza

User: how about pasta
Bot: Yes, pasta is good.

Chatbot session ended.


## Language Translations

In [11]:
# Step 1: Install dependencies
#!pip install transformers sentencepiece torch

In [12]:
# Step 2: Import libraries
from transformers import MarianMTModel, MarianTokenizer

# Step 3: Choose source and target languages
# Example: English to French
src_lang = "en"
tgt_lang = "fr"

model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"

# Step 4: Load tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/opt/anaconda3/envs/AAI/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [13]:
# Step 5: Input text for translation
input_text = ["Hello, how are you today?"]

# Step 6: Tokenize input
inputs = tokenizer(input_text, return_tensors="pt", padding=True)

# Step 7: Generate translation
translated = model.generate(**inputs)

# Step 8: Decode and display
translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

print(f"Input: {input_text[0]}")
print(f"Translated: {translated_text[0]}")


Input: Hello, how are you today?
Translated: Bonjour, comment allez-vous aujourd'hui ?


In [14]:
while True:
    # Take user input
    text = input("You (English): ")
    if text.lower() == "quit":
        print("🔚 Translation session ended.")
        break

    # Tokenize input text
    inputs = tokenizer([text], return_tensors="pt", padding=True)

    # Generate translation
    translated = model.generate(**inputs, num_beams=5, max_length=100)

    # Decode translated text
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

    # Show result
    print(f"Bot (French): {translated_text}\n")

Bot (French): Bonjour mon nom est Sahil

Bot (French): Je vis en Inde

🔚 Translation session ended.


## Summarization

In [2]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))